In [33]:
import easyocr
import cv2
import numpy as np
from regex_matching import *

path = r'./generated_labels/'
path = r'./temp/'
img_name = "label_0_base.png"
img_name = "tuna_crop.jpg"

def pretty_print_easyocr(results):
    """
    Pretty-print EasyOCR output.
    Expected format: [
        ([ [x1, y1], [x2, y2], [x3, y3], [x4, y4] ], 'text', confidence),
        ...
    ]
    """
    print(f"{'Index':<6} {'Bounding Box (x,y)':<45} {'Text':<20} {'Confidence':<10}")
    print("-" * 85)
    
    for i, (bbox, text, conf) in enumerate(results):
        # Flatten bounding box for compact display
        bbox_str = ", ".join([f"({int(x)},{int(y)})" for x, y in bbox])
        print(f"{i:<6} {bbox_str:<45} {text:<20} {conf:.6f}")

reader = easyocr.Reader(['en'], gpu=False)

# Run OCR
results = reader.readtext(path + img_name)

#pretty_print_easyocr(results)

# Load the original image
image = cv2.imread(path + img_name)

# Loop through OCR results
for (bbox, text, prob) in results:
    # bbox is [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
    pts = [tuple(map(int, point)) for point in bbox]
    
    # Draw the bounding box (polygon)
    cv2.polylines(image, [np.array(pts, dtype=np.int32)], True, (0, 255, 0), 2)
    
    # Put the text above the top-left corner of the bbox
    cv2.putText(image, text, pts[0], cv2.FONT_HERSHEY_SIMPLEX, 
                2, (75, 250, 55), 3, cv2.LINE_AA)

print("Tekst u linijama", easyocr_to_lines(results))

final_res = extract_nutrients(easyocr_to_lines(results), nutrient_aliases)

# Save the result
output_path = path + "detected_text_tuna.png"
cv2.imwrite(output_path, image)

print(final_res)
print(f"Saved result with bounding boxes to {output_path}")



Using CPU. Note: This module is much faster with a GPU.
c:\Aleksa_sve\master\master_rad_py\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Tekst u linijama ['konzerva prosecne nutritivne 0', 'himaiuri', 'ssboji; stipjack / vrednosti na 100 g neto mase:', '3', 'ribu', 'de', 'energija 511 kj / 121 kcal', 'jamisuva estu, masti 1,49 ~', 'od kojih zasicene 3 ', 'masne kiseline 0,69', 'uglieniojidraeceri', 'og', 'fa lgason.cz', 'proteini 27 19', '1119 so', 'je.min,']
Calories: 511 kcal
Total fat: 1.5 g
  Saturated fat: No value
Carbohydrate: No value
  Dietary fiber: No value
  Sugars: No value
Protein: 27.0 g
Saved result with bounding boxes to ./temp/detected_text_tuna.png


In [34]:
easyocr_to_lines(results)
final_res
final_res = extract_nutrients(easyocr_to_lines(results), nutrient_aliases)


In [30]:
match = re.search(r"(\d+[.,]?\d*)\s*(kcal|kj|g|mg|mcg|µg)?", easyocr_to_lines(results)[6])
print(match)
if match:
    value = float(match.group(1).replace(",", "."))
    
    if ("kj" in match.group()):
        value = int(value/4.2)
    print(value)

final_res

<re.Match object; span=(9, 15), match='511 kj'>
121


NutritionLabelData(calories=511.0, total_fat=1.49, saturated_fat=None, carbohydrates=None, dietary_fiber=None, sugars=None, protein=27.0)

In [25]:
easyocr_to_lines(results)[6]
match

<re.Match object; span=(9, 15), match='511 kj'>